In [1]:
%load_ext autoreload

In [2]:
import numpy as np
from functools import partial
import visualization_utils as viz_utils
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
from pathlib import Path
import os

In [127]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix,MeshcatVisualizer, StartMeshcat
import pydrake.symbolic as sym
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions, ScsSolver
from pydrake.all import PointCloud, MeshcatVisualizerParams, Role, HalfSpace, CoulombFriction
from pydrake.polynomial import Polynomial as PolynomialCommon
import time

from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import HPolyhedron, Hyperellipsoid
from pydrake.geometry.optimization_dev import CspaceFreePath


In [4]:
import logging
drake_logger = logging.getLogger("drake")
drake_logger.setLevel(logging.DEBUG)

# Build and set up the visualization the plant and the visualization of the C-space obstacle

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [ ]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
model_file = FindResourceOrThrow("drake/manipulation/models/iiwa_description/iiwa7/iiwa7_with_box_collision.sdf")
box_file = FindResourceOrThrow("drake/C_Iris_Examples/assets/shelves.sdf")


parser = Parser(plant)

friction = CoulombFriction(1,1)
ground = plant.RegisterCollisionGeometry(
    plant.world_body(), HalfSpace.MakePose(np.array([0,0,1.]), np.array([0,0,0])),
    HalfSpace(), "collision", friction)

models = []
models.append(Parser(plant, scene_graph).AddModelFromFile(model_file))
models.append(Parser(plant, scene_graph).AddModelFromFile(box_file))

sp = 0.0
x_fac = 0.5
locs = [ [0,0,0], 
        [x_fac, 1.4*sp,0.4], [x_fac,-1.4*sp,0.4], [-x_fac,-1.4*sp,0.4], [-x_fac,1.4,0.4], 
        [0.0 ,0 , 0.95], [0.0 ,0 , -0.05]] 
idx = 0
for model in models:
    plant.WeldFrames(ground.body_frame(), plant.get_body(plant.GetBodyIndices(model)[0]).body_frame(),
                     RigidTransform(locs[idx]))
    idx +=1


    
plant.Finalize()


        
# construct the RationalForwardKinematics of this plant. This object handles the
# computations for the forward kinematics in the tangent-configuration space
Ratfk = RationalForwardKinematics(plant)

# the point about which we will take the stereographic projections
q_star = np.zeros(plant.num_positions())

do_viz = True

meshcat = StartMeshcat()
meshcat.Delete()

visualizer_params = MeshcatVisualizerParams()
visualizer_params.role = Role.kProximity
visualizer = MeshcatVisualizer.AddToBuilder(
    builder, scene_graph, meshcat)
diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()
diagram.ForcedPublish(diagram_context)

In [6]:
#compute limits in s-space
limits_s = []
for q in [plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits()]:
    limits_s.append(Ratfk.ComputeSValue(np.array(q), q_star))
limits_s = np.array(limits_s)

Time to construct line certifier = 7.623325347900391s


DEBUG:drake:Meshcat connection opened from 0000:0000:0000:0000:0000:0000:0000:0001


In [8]:
from pydrake.all import InverseKinematics
ik = InverseKinematics(plant, diagram.GetMutableSubsystemContext(plant, diagram_context))
min_dist = 1e-5
collision_constraint = ik.AddMinimumDistanceConstraint(
            min_dist, 1e-5)

def check_collision_q_by_ik(q, min_dist=1e-5):
    if np.all(q >= plant.GetPositionLowerLimits()) and \
            np.all(q <= plant.GetPositionUpperLimits()):
        return 1 - 1 * \
            float(collision_constraint.evaluator().CheckSatisfied(q, min_dist))
    else:
        return 1
def check_collision_s_by_ik(s, min_dist=1e-5):
    s = np.array(s)
    q = Ratfk.ComputeQValue(s, q_star)
    return check_collision_q_by_ik(q, min_dist)

In [9]:
prm_important_samples_q = np.array([
    [-0.47, -0.19, 0.34, -1.89, 0.68, 0.18, 0],
    [-2.13, -1.44,  1.53, -1.9 , -2.97,  0.6 ,  0.22],
    [2.22, -2.09, -1.61, -1.61, -2.87,  0.64,  3.05]
]) 

prm_import_samples_s = [ Ratfk.ComputeSValue(x, q_star) for x in prm_important_samples_q]

In [10]:
# draw prm
import prm
from pydrake.all import Rgba

# limits = [np.array(t_low), np.array(q_high)]



def collision(pos, col_func_handle):
    return col_func_handle(pos)

def collision_bad(pos, col_func_handle):
    return 1-col_func_handle(pos)

prm_col_fn_handle = partial(collision, col_func_handle = check_collision_s_by_ik)
prm_col_fn_handle_bad = partial(collision_bad, col_func_handle = check_collision_s_by_ik)

PRM = prm.PRM( 
            limits_s,
            num_points = 100,
            col_func_handle = prm_col_fn_handle,
            num_neighbours = 3, 
            dist_thresh = 10,
            num_col_checks = 10,
            max_it = 1e3,
            initial_points = prm_import_samples_s,
            verbose = True,
            plotcallback = None
            )

# PRM_bad = prm.PRM( 
#             limits_t,
#             num_points = 2000,
#             col_func_handle = prm_col_fn_handle_bad,
#             num_neighbours = 5, 
#             dist_thresh = .5,
#             num_col_checks = 10,
#             verbose = True,
#             plotcallback = None
#             )

# PRM.add_start_end(start, target)
# PRM.plot()
tot_num_edges = len(PRM.adjacency_list)* len(PRM.adjacency_list[0])
print(tot_num_edges)

[PRM] Samples 0
[PRM] Samples 30
[PRM] Samples 60
[PRM] Samples 90
[PRM] Nodes connected: 0
[PRM] Nodes connected: 20
[PRM] Nodes connected: 40
[PRM] Nodes connected: 60
[PRM] Nodes connected: 80
300


In [11]:
body = plant.GetBodyByName("iiwa_link_7")
plant_context = diagram.GetMutableSubsystemContext(plant, diagram_context)
def visualize_task_space_trajectory(segment_start, segment_end,
                                    body, color = Rgba(0,1,0,0.5),
                                    path_size = 0.01, num_points = 100, path = "prm/seg"):
    s_waypoints = np.linspace(segment_start, segment_end, num_points)
    points = []
    for s in s_waypoints:
        q = Ratfk.ComputeQValue(s, q_star)
        plant.SetPositions(plant_context, q)
        points.append(plant.EvalBodyPoseInWorld(plant_context, body).translation())
    points = np.array(points)
    pc = PointCloud(len(points))
    pc.mutable_xyzs()[:] = points.T
    meshcat.SetObject(path, pc, point_size = path_size, rgba=color)


def plot_prm(PRM, body, color = Rgba(0,1,0,0.5),
                                    path_size = 0.01, num_points = 1000, prefix = "prm"):
    meshcat.Delete(prefix)
    for idx, (s0, s1) in enumerate(PRM.prm_pairs):
        visualize_task_space_trajectory(s0, s1, body, color,
                                        path_size, num_points, 
                                        f"{prefix}/seg_{idx}")
def plot_prm_nodes(PRM, body, color = Rgba(0,0,1,1),
                                    point_size = 0.05, prefix = "prm_nodes"):
    meshcat.Delete(prefix)
    points = []
    for idx, s in enumerate(PRM.nodes):
        q = Ratfk.ComputeQValue(s, q_star)
        plant.SetPositions(plant_context, q)
        points.append(plant.EvalBodyPoseInWorld(plant_context, body).translation())
    points = np.array(points)
    pc = PointCloud(len(points))
    pc.mutable_xyzs()[:] = points.T
    meshcat.SetObject(prefix, pc, point_size = point_size, rgba=color)
    
plot_prm(PRM, body)
plot_prm_nodes(PRM, body)

## Set up the sliders so we can move the plant around manually

In [12]:
sliders = []
for i in range(plant.num_positions()):
    sliders.append(widgets.FloatSlider(min=plant.GetPositionLowerLimits()[i],
                                       max=plant.GetPositionUpperLimits()[i], 
                                       value=0, description=f'q{i}'))
# sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = np.zeros(plant.num_positions())
def handle_slider_change(change, idx):
    q[idx] = change['new']
    plant.SetPositions(plant_context, q)
    diagram.ForcedPublish(diagram_context)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


# visualizer.jupyter_cell()

Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


In [14]:
def make_line_polys(PRM):
    polys = np.empty(shape=(plant.num_positions(), len(PRM.prm_pairs)), dtype = object)
    for i, (s0, s1) in enumerate(PRM.prm_pairs):
        for j in range(plant.num_positions()):
            polys[j,i] = PolynomialCommon(np.array([s0[j],s1[j]-s0[j]]))# PolynomialCommon(np.array([s1[j],s0[j]-s1[j]]))
            if i == 0 and j == 0:
                print(polys[j,i].EvaluateUnivariate(0))
                print(s0[j])
                print()
                print(polys[j,i].EvaluateUnivariate(1))
                print(s1[j])
                print()
    return polys

path_safe = make_line_polys(PRM)
# path_unsafe = make_line_polys(PRM_bad)
    

-3.658497391327363
-3.658497391327363

-2.5392052873571522
-2.5392052873571522



In [21]:
# The object we will use to perform our certification.
t0 = time.time()
cspace_free_path = CspaceFreePath(plant, scene_graph, q_star, maximum_path_degree = 1, plane_order = 4)
t1 = time.time()
print(f"Time to construct line certifier = {t1-t0}s")


cert_options = CspaceFreePath.FindSeparationCertificateGivenPathOptions()
cert_options.terminate_segment_certification_at_failure = False

cert_options.num_threads = -1
cert_options.verbose = False
cert_options.solver_id = MosekSolver.id()
cert_options.solver_options = SolverOptions()
cert_options.terminate_path_certification_at_failure = False


Time to construct line certifier = 23.502044916152954s


In [22]:
t0 = time.time()
ret = cspace_free_path.FindSeparationCertificateGivenPath(path_safe, set(), cert_options)
t1 = time.time()
print(f"Certification of safe PRM in {t1-t0}s")

Certification of safe PRM in 140.31892371177673s


In [26]:
def plot_pair_safety(pairs, is_safe_vec, body,
                                    path_size = 0.01, num_points = 1000, prefix = "prm"):
    meshcat.Delete(f"{prefix}_safe")
    meshcat.Delete(f"{prefix}_unsafe")
    for idx, (s0, s1) in enumerate(pairs):
        if is_safe_vec[idx]:
            visualize_task_space_trajectory(s0, s1, body, Rgba(0,1,0,0.5),
                                            path_size, num_points, 
                                            f"{prefix}_safe/seg_{idx}")
        else:
            visualize_task_space_trajectory(s0, s1, body, Rgba(1,0,0,0.5),
                                            path_size, num_points, 
                                            f"{prefix}_unsafe/seg_{idx}")
plot_pair_safety(PRM.prm_pairs, ret[0], body)

In [53]:
unsafe_inds = np.where(np.logical_not(ret))[1]

In [55]:
unsafe_traj = path_safe[:,unsafe_inds]

In [57]:
unsafe_traj[:,0]

array([<pydrake.polynomial.Polynomial object at 0x7f38c77dab70>,
      dtype=object)

In [119]:
def animate_traj_s(traj, steps, sleep_time = 0.1):
    # loop
    idx = 0
    going_fwd = True
    time_points = np.linspace(0, traj.end_time(), steps)
    frame_count = 0
    meshcat.StartRecording()
    for frame_count, t in enumerate(time_points):
        t0 = time.time()
        s = traj.value(t)
        q = Ratfk.ComputeQValue(s, q_star)
        plant.SetPositions(plant_context, q)
        diagram_context.SetTime(frame_count *sleep_time)
        diagram.ForcedPublish(diagram_context)
        
        in_collision = check_collision_q_by_ik(q)
        if in_collision:
            print("COLLISION DETECTED")
        t1 = time.time()
        pause = sleep_time - (t1 - t0)
        if pause > 0:
            time.sleep(pause)
    meshcat.StopRecording()
    meshcat.PublishRecording()

In [120]:
from pydrake.all import PiecewisePolynomial
traj = PiecewisePolynomial(list(unsafe_traj[:,0][np.newaxis,:]), [0,1])
print(traj.rows())
print(traj.cols())

7
1


In [122]:
i = 0
animate_traj_s(traj, 1000, 0.1)

COLLISION DETECTED
COLLISION DETECTED
COLLISION DETECTED
COLLISION DETECTED
COLLISION DETECTED
COLLISION DETECTED
COLLISION DETECTED
COLLISION DETECTED
COLLISION DETECTED
COLLISION DETECTED
COLLISION DETECTED
COLLISION DETECTED
